# Object Detection
This guide demonstrates the seperate full workflow to run PicoDetV2 model on Arm® Ethos™-U85 processor.  

## PicoDetV2

In [ ]:
# create virtual environment with Python 3.10
!python3.10 -m venv picodetv2
!source picodetv2/bin/activate
!cd picodetv2

In [ ]:
# Install dependencies
!cat <<EOF > picodetv2_req.txt
asttokens==3.0.0
coloredlogs==15.0.1
decorator==5.2.1
exceptiongroup==1.3.0
executing==2.2.0
flatbuffers==25.2.10
humanfriendly==10.0
jedi==0.19.2
matplotlib-inline==0.1.7
mpmath==1.3.0
paddlelite==2.13rc0
parso==0.8.4
pexpect==4.9.0
pip==25.2
prompt_toolkit==3.0.51
ptyprocess==0.7.0
pure_eval==0.2.3
setuptools==78.1.1
stack-data==0.6.3
traitlets==5.14.3
wcwidth==0.2.13
wheel==0.45.1
EOF
!pip install -r ./picodetv2_req.txt

In [ ]:
# Download example source code
!git clone https://github.com/Arm-Examples/Paddle-on-Ethos-U.git
!cd Paddle-on-Ethos-U
git lfs pull
# Configure inference environment
!bash install.sh

In [ ]:
# Download picodetv2 model
!wget -O ./model_zoo/PicoDetV2_infer_int8/picodetv2_relu6_coco_no_fuse.tar.gz https://paddlelite-demo.bj.bcebos.com/Paddle-Lite-Demo/models/picodetv2_relu6_coco_no_fuse.tar.gz
!cd ./model_zoo/PicoDetV2_infer_int8
!tar -xvzf picodetv2_relu6_coco_no_fuse.tar.gz

In [ ]:
# Convert model into .nb format
!paddle_lite_opt --model_dir=./picodetv2_relu6_coco_no_fuse --optimize_out_type=naive_buffer --optimize_out=./picodetv2_relu6_coco_no_fuse --quant_model=true --quant_type=QUANT_INT8 --valid_targets=arm # The model file "picodetv2_relu6_coco_no_fuse.nb" will be generated under current directory.

In [ ]:
# Convert .nb model into IR file (.json file)
!cd ../..
!python ./readnb/write_model.py --model_path ./model_zoo/PicoDetV2_infer_int8/picodetv2_relu6_coco_no_fuse.nb --out_dir . # "g_picodetv2_relu6_coco_no_fuse.json" is generated under the same directory with input model file
# Modify the IR file with patch quickly. You could also do these modification maunally.
!patch -p0 model_zoo/PicoDetV2_infer_int8/g_picodetv2_relu6_coco_no_fuse.json < readnb/test_asset/picodet/g_picodetv2.patch

In [ ]:
# Run inference
!bash paddle_verify.sh -m picodetv2 -p ./model_zoo/PicoDetV2_infer_int8/demo.jpg